This Python Jupyter Notebook provides updates to some of the initial information imported in: https://github.com/danielunderwood2292/Public/blob/main/Elections/B_Insert_Initial_Info.ipynb

In [1]:
#Import required packages
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import pandas as pd

In [2]:
#Connect to database 'UK_General_Election' using SQlAlchemy
connection_str = "DRIVER={SQL SERVER};SERVER=DANZPOOTA;DATABASE=UK_General_Election;TRUSTED_CONNECTION=YES"
params = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
conn = engine.connect()

In [3]:
UpdatePollsters = """
UPDATE Pollsters
SET DefaultRegionType = 'GB'
WHERE PollsterName = 'Focaldata' OR PollsterName = 'Panelbase'

UPDATE Pollsters
SET DefaultRegionType = 'ITL1Region'
WHERE PollsterName = 'BMG' OR PollsterName = 'Omnisis' OR PollsterName = 'Savanta'
OR PollsterName = 'Redfield & Wilton' OR PollsterName = 'JL Partners' OR PollsterName = 'FindOutNow'

UPDATE Pollsters
SET DefaultRegionType = 'Survation'
WHERE PollsterName = 'Opinium'

UPDATE Pollsters
SET DefaultRegionType = 'YouGov'
WHERE PollsterName = 'PeoplePolling'

UPDATE Pollsters
SET DefaultRegionType = 'Nation'
WHERE PollsterName = 'LucidTalk'
"""

In [4]:
engine.execute(UpdatePollsters)

In [5]:
#Read table back out from the database to confirm the values have been entered
db_pollsters_df = pd.read_sql('SELECT * From Pollsters',conn)
db_pollsters_df

,PollsterName,DefaultRegionType
0,BMG,ITL1Region
1,Deltapoll,Survation
2,FindOutNow,ITL1Region
3,Focaldata,GB
4,ICM,ICM
5,Ipsos,YouGov
6,JL Partners,ITL1Region
7,Kantar Public,GB
8,LucidTalk,Nation
9,More in Common,ITL1Region


In [6]:
#Location of file to be used
filename = "C:\\Users\\danmu\\Documents\\Elections\\2024_Python\\Initial_Data.xlsx"
NewCandidates_df = pd.read_excel(filename, sheet_name='NewCandidates')
NewCandidates_df.to_sql('Candidates', conn, if_exists='append', index=False)
db_NewCandidates_df = pd.read_sql('SELECT * From Candidates WHERE PreviousVotes = 0',conn)
db_NewCandidates_df.tail()

,CandidateID,Constituency,Party,SittingMP,FormerMP,FirstName,Surname,Gender,PreviousVotes,PreviousShare,Standing
822,Ynys MonGreen,Ynys Mon,Green,False,False,None,None,None,0,0.0,True
823,Ynys MonLD,Ynys Mon,LD,False,False,None,None,None,0,0.0,True
824,Ynys MonOther,Ynys Mon,Other,False,False,None,None,None,0,0.0,True
825,York OuterGreen,York Outer,Green,False,False,None,None,None,0,0.0,True
826,York OuterReform,York Outer,Reform,False,False,None,None,None,0,0.0,True


In [7]:
conn.close()